In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc
from tqdm import tqdm
from sklearn import preprocessing
from scipy.stats import entropy, kurtosis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [2]:
import gc
gc.collect()

31

In [3]:
path = 'D:/data/'
train_jet = pd.read_feather(path+'jet_complex_data/complex_train_R04_jet.feather')
train_particle = pd.read_feather(path+'jet_complex_data/complex_train_R04_particle.feather')
test_jet = pd.read_feather(path+'jet_complex_data/complex_test_R04_jet.feather')
test_particle = pd.read_feather(path+'jet_complex_data/complex_test_R04_particle.feather')

In [4]:
#data: jet_id级样本
data = pd.concat([train_jet, test_jet], sort=False)

In [5]:
data['jet_energy/num'] = data['jet_energy'] / data['number_of_particles_in_this_jet']
data['jet_mass'].loc[data['jet_mass']<0] = 0
data['jet_mass/num'] = data['jet_mass'] / data['number_of_particles_in_this_jet']
data['jet_px/num'] = np.abs(data['jet_px']) / data['number_of_particles_in_this_jet']
data['jet_py/num'] = np.abs(data['jet_py']) / data['number_of_particles_in_this_jet']
data['jet_pz/num'] = np.abs(data['jet_pz']) / data['number_of_particles_in_this_jet']



data['jet_|p|'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py'])+np.square(data['jet_pz']))
data['jet_pt'] = np.sqrt(np.square(data['jet_px'])+np.square(data['jet_py']))

data['jet_phi_cos'] = data['jet_px'] / data['jet_pt']
data['jet_eta_sinh'] = data['jet_pz'] / data['jet_pt']
data['jet_phi'] = np.arccos(data['jet_phi_cos'])
data['jet_eta'] = np.arcsinh(data['jet_eta_sinh'])

data['jet_theta_cos'] = np.tanh(data['jet_eta'])
data['jet_theta'] = np.arccos(data['jet_theta_cos'])

for col in ['jet_phi_cos', 'jet_eta_sinh', 'jet_theta_cos']:
    data[col+'_abs'] = data[col].abs()

data['jet_energy_differ'] = data['jet_energy'] - data['jet_|p|']
data['jet_energy_differ/num'] = data['jet_energy_differ'] / data['number_of_particles_in_this_jet']

data['jet_mass/pt'] = data['jet_mass'] / data['jet_pt']

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
#ans: event_id级样本
ans = data.groupby('event_id', as_index=False)['label'].agg({'label':np.mean})
ans.columns = ['event_id', 'label']
count = data.groupby('event_id', as_index=False)['jet_px'].count()
count.columns = ['event_id', 'count']
ans['count'] = count['count']
ans

,event_id,label,count
0,00000fdcf3f1454f962edf1cc6a2f64b,5.0,3
1,0000103e24d74ddd94a00a30d476a301,1.0,2
2,0000113f88864edebf628441e8b962ec,4.0,2
3,0000571f2c9b4e90a5701b32a1b2d26f,5.0,2
4,0000936dc7ee482b817758c25b567378,NaN,2
...,...,...,...
548092,fffee9ef58654385b65e3b3997cdbbd1,21.0,3
548093,fffeec406f5441ebb0904190622dd9f4,5.0,2
548094,ffff46d0d16746babfb51e1f982aea81,1.0,2
548095,ffff51e8086e4295bf86c502829bab3d,4.0,2


In [7]:
def count_all(df_group, df_merge, base_column, count_columns):
    gp = df_group.groupby(base_column, as_index=False)
    for count_column in tqdm(count_columns):
        tp = gp[count_column].agg({
             base_column+'_'+count_column+'_mean': 'mean', 
             base_column+'_'+count_column+'_max': 'max', 
             base_column+'_'+count_column+'_min': 'min', 
             base_column+'_'+count_column+'_sum': 'sum', 
             base_column+'_'+count_column+'_std': 'std', 
#              base_column+'_'+count_column+'_skew': 'skew', 
#              base_column+'_'+count_column+'_kurt': kurtosis, 
#              base_column+'_'+count_column+'_q1': lambda x: np.quantile(x, q=0.25), 
#              base_column+'_'+count_column+'_q3': lambda x: np.quantile(x, q=0.75), 
        })
        df_merge = df_merge.merge(tp, on=base_column, how='left')
    return df_merge

# temp = pd.DataFrame({'1':[1,2,2,3,3,3,4,4,4,4], '2':[1,2,3,4,5,6,7,8,9,10]})
# temp
# count_all(temp, temp, '1', '2')

# particle->jet

In [8]:
#data_particle: particle_id级样本
data_particle = pd.concat([train_particle, test_particle], sort=False)

In [9]:
data_particle = pd.get_dummies(data_particle, columns=['particle_category'])

In [10]:
data_particle['particle_|p|'] = np.sqrt(np.square(data_particle['particle_px'])+np.square(data_particle['particle_py'])+np.square(data_particle['particle_pz']))
data_particle['particle_pt'] = np.sqrt(np.square(data_particle['particle_px'])+np.square(data_particle['particle_py']))

data_particle['particle_phi_cos'] = data_particle['particle_px'] / data_particle['particle_pt']
data_particle['particle_eta_sinh'] = data_particle['particle_pz'] / data_particle['particle_pt']
data_particle['particle_phi'] = np.arccos(data_particle['particle_phi_cos'])
data_particle['particle_eta'] = np.arcsinh(data_particle['particle_eta_sinh'])

data_particle['particle_theta_cos'] = np.tanh(data_particle['particle_eta'])
data_particle['particle_theta'] = np.arccos(data_particle['particle_theta_cos'])

for col in ['particle_phi_cos', 'particle_eta_sinh', 'particle_theta_cos']:
    data_particle[col+'_abs'] = data_particle[col].abs()
    
data_particle['particle_energy_differ'] = data_particle['particle_energy'] - data_particle['particle_|p|']

data_particle['particle_mass/pt'] = data_particle['particle_mass'] / data_particle['particle_pt']

In [11]:
data_particle = pd.merge(data_particle, data[['event_id', 'jet_id', 'jet_phi', 'jet_eta', 'jet_pt', 'jet_energy', 'jet_theta']], on ='jet_id', how='left')

In [12]:
data_particle['particle_phi_rel'] = data_particle['particle_phi'] - data_particle['jet_phi']
data_particle['particle_eta_rel'] = data_particle['particle_eta'] - data_particle['jet_eta']
data_particle['particle_theta_rel'] = data_particle['particle_theta'] - data_particle['jet_theta']
data_particle['particle_pt_rel'] = data_particle['particle_pt'] / data_particle['jet_pt']
data_particle['particle_energy_rel'] =  data_particle['particle_energy'] / data_particle['jet_energy']
data_particle['particle_R'] = np.sqrt(data_particle['particle_phi_rel']**2 + data_particle['particle_eta_rel']**2)

In [13]:
data_particle = data_particle.drop(['jet_phi', 'jet_eta', 'jet_pt', 'jet_energy', 'jet_theta'], axis=1)

In [14]:
data.columns.to_list()
data_particle.columns.to_list()

['jet_id',
 'number_of_particles_in_this_jet',
 'jet_px',
 'jet_py',
 'jet_pz',
 'jet_energy',
 'jet_mass',
 'event_id',
 'label',
 'jet_energy/num',
 'jet_mass/num',
 'jet_px/num',
 'jet_py/num',
 'jet_pz/num',
 'jet_|p|',
 'jet_pt',
 'jet_phi_cos',
 'jet_eta_sinh',
 'jet_phi',
 'jet_eta',
 'jet_theta_cos',
 'jet_theta',
 'jet_phi_cos_abs',
 'jet_eta_sinh_abs',
 'jet_theta_cos_abs',
 'jet_energy_differ',
 'jet_energy_differ/num',
 'jet_mass/pt']

['particle_px',
 'particle_py',
 'particle_pz',
 'particle_energy',
 'particle_mass',
 'jet_id',
 'particle_category_-2212',
 'particle_category_-2112',
 'particle_category_-321',
 'particle_category_-211',
 'particle_category_-13',
 'particle_category_-11',
 'particle_category_11',
 'particle_category_13',
 'particle_category_22',
 'particle_category_130',
 'particle_category_211',
 'particle_category_321',
 'particle_category_2112',
 'particle_category_2212',
 'particle_|p|',
 'particle_pt',
 'particle_phi_cos',
 'particle_eta_sinh',
 'particle_phi',
 'particle_eta',
 'particle_theta_cos',
 'particle_theta',
 'particle_phi_cos_abs',
 'particle_eta_sinh_abs',
 'particle_theta_cos_abs',
 'particle_energy_differ',
 'particle_mass/pt',
 'particle_phi_rel',
 'particle_eta_rel',
 'particle_theta_rel',
 'particle_pt_rel',
 'particle_energy_rel',
 'particle_R']

In [15]:
#ans_particle: jet_id级样本
ans_particle = data_particle.groupby('jet_id', as_index=False)['particle_category_-2212',
 'particle_category_-2112',
 'particle_category_-321',
 'particle_category_-211',
 'particle_category_-13',
 'particle_category_-11',
 'particle_category_11',
 'particle_category_13',
 'particle_category_22',
 'particle_category_130',
 'particle_category_211',
 'particle_category_321',
 'particle_category_2112',
 'particle_category_2212'].sum()
ans_particle

,jet_id,particle_category_-2212,particle_category_-2112,particle_category_-321,particle_category_-211,particle_category_-13,particle_category_-11,particle_category_11,particle_category_13,particle_category_22,particle_category_130,particle_category_211,particle_category_321,particle_category_2112,particle_category_2212
0,000006d8f2c143e39d40961834db96e6,0,0,1,4,0,0,0,0,13,0,2,1,0,0
1,00005b43827d451fbf43b7a6b8c90099,0,1,0,4,0,0,0,0,0,1,3,0,0,1
2,00006de576aa41ca93b2549401436baf,0,0,1,3,0,0,0,0,5,1,1,2,0,0
3,000070820a23425aa2881cc5d5041735,0,0,0,10,0,0,0,0,23,0,11,1,0,0
4,000076b478704551bc7439b81110fdfb,0,0,1,5,0,0,0,0,17,0,5,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672499,ffffbee743444ee3974e6679059d4eb5,0,0,0,4,0,0,0,0,5,2,3,0,0,0
1672500,ffffded0be154db080c3e584a51046f6,0,0,1,7,0,0,0,0,3,0,4,2,0,0
1672501,ffffe566099149af8855513c5c73621c,0,0,1,2,0,0,0,0,15,2,3,0,0,0
1672502,fffff357e4c84b61a2a471ea2753c463,1,0,0,1,0,0,0,0,8,0,3,0,1,0


In [16]:
data_particle_col = data_particle.columns.to_list()
data_particle_col.remove('jet_id', 'event_id')
ans_particle = count_all(data_particle, ans_particle, 'jet_id', data_particle_col)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [03:07<00:00,  7.83s/it]


In [17]:
data = data.merge(ans_particle, on='jet_id', how='left')
del ans_particle

# jet->event

In [18]:
gc.collect()

40

In [19]:
data_col = data.columns.to_list()
for col in data_col:
    if 'std' in col or 'skew' in col or 'kurt' in col:
        data[col] = data[col].fillna(0)
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1672504 entries, 0 to 1672503
Data columns (total 162 columns):
jet_id                                1672504 non-null object
number_of_particles_in_this_jet       1672504 non-null int64
jet_px                                1672504 non-null float64
jet_py                                1672504 non-null float64
jet_pz                                1672504 non-null float64
jet_energy                            1672504 non-null float64
jet_mass                              1672504 non-null float64
event_id                              1672504 non-null object
label                                 1134555 non-null float64
jet_energy/num                        1672504 non-null float64
jet_mass/num                          1672504 non-null float64
jet_px/num                            1672504 non-null float64
jet_py/num                            1672504 non-null float64
jet_pz/num                            1672504 non-null float64
jet_|p| 

In [20]:
data_col.remove('jet_id')
data_col.remove('event_id')
data_col.remove('label')
print(len(data_col))
print(data_col)

159
['number_of_particles_in_this_jet', 'jet_px', 'jet_py', 'jet_pz', 'jet_energy', 'jet_mass', 'jet_energy/num', 'jet_mass/num', 'jet_px/num', 'jet_py/num', 'jet_pz/num', 'jet_|p|', 'jet_pt', 'jet_phi_cos', 'jet_eta_sinh', 'jet_phi', 'jet_eta', 'jet_theta_cos', 'jet_theta', 'jet_phi_cos_abs', 'jet_eta_sinh_abs', 'jet_theta_cos_abs', 'jet_energy_differ', 'jet_energy_differ/num', 'jet_mass/pt', 'particle_category_-2212', 'particle_category_-2112', 'particle_category_-321', 'particle_category_-211', 'particle_category_-13', 'particle_category_-11', 'particle_category_11', 'particle_category_13', 'particle_category_22', 'particle_category_130', 'particle_category_211', 'particle_category_321', 'particle_category_2112', 'particle_category_2212', 'jet_id_particle_px_mean', 'jet_id_particle_px_max', 'jet_id_particle_px_min', 'jet_id_particle_px_sum', 'jet_id_particle_px_std', 'jet_id_particle_py_mean', 'jet_id_particle_py_max', 'jet_id_particle_py_min', 'jet_id_particle_py_sum', 'jet_id_part

In [21]:
ans = count_all(data, ans, 'event_id', data_col)

100%|████████████████████████████████████████████████████████████████████████████████| 159/159 [04:05<00:00,  1.55s/it]


In [22]:
ans = ans.reset_index(drop=True)
ans.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 798 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float64
count                                               548097 non-null int64
event_id_number_of_particles_in_this_jet_mean       548097 non-null float64
event_id_number_of_particles_in_this_jet_max        548097 non-null int64
event_id_number_of_particles_in_this_jet_min        548097 non-null int64
event_id_number_of_particles_in_this_jet_sum        548097 non-null int64
event_id_number_of_particles_in_this_jet_std        545759 non-null float64
event_id_jet_px_mean                                548097 non-null float64
event_id_jet_px_max                                 548097 non-null float64
event_id_jet_px_min                                 548097 non-null float64
event_id_jet_px_sum                                 548097 no

In [23]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32) #if not to_feather: np.float16
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [24]:
ans_col = ans.columns.to_list()
for col in ans_col:
    if 'std' in col or 'skew' in col or 'kurt' in col:
        ans[col] = ans[col].fillna(0)
ans.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 798 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float64
count                                               548097 non-null int64
event_id_number_of_particles_in_this_jet_mean       548097 non-null float64
event_id_number_of_particles_in_this_jet_max        548097 non-null int64
event_id_number_of_particles_in_this_jet_min        548097 non-null int64
event_id_number_of_particles_in_this_jet_sum        548097 non-null int64
event_id_number_of_particles_in_this_jet_std        548097 non-null float64
event_id_jet_px_mean                                548097 non-null float64
event_id_jet_px_max                                 548097 non-null float64
event_id_jet_px_min                                 548097 non-null float64
event_id_jet_px_sum                                 548097 no

In [25]:
ans = reduce_mem_usage(ans)
ans.info(verbose=True, null_counts=True)

Mem. usage decreased to 1598.96 Mb (28.7% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 798 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float32
count                                               548097 non-null int8
event_id_number_of_particles_in_this_jet_mean       548097 non-null float32
event_id_number_of_particles_in_this_jet_max        548097 non-null int8
event_id_number_of_particles_in_this_jet_min        548097 non-null int8
event_id_number_of_particles_in_this_jet_sum        548097 non-null int16
event_id_number_of_particles_in_this_jet_std        548097 non-null float32
event_id_jet_px_mean                                548097 non-null float32
event_id_jet_px_max                                 548097 non-null float32
event_id_jet_px_min                                 548097 non-null float32
event_id_je

In [26]:
ans.to_feather('E:/jet_complex_data/data_new.feather')

In [27]:
ans.head()

,event_id,label,count,event_id_number_of_particles_in_this_jet_mean,event_id_number_of_particles_in_this_jet_max,event_id_number_of_particles_in_this_jet_min,event_id_number_of_particles_in_this_jet_sum,event_id_number_of_particles_in_this_jet_std,event_id_jet_px_mean,event_id_jet_px_max,event_id_jet_px_min,event_id_jet_px_sum,event_id_jet_px_std,event_id_jet_py_mean,event_id_jet_py_max,event_id_jet_py_min,event_id_jet_py_sum,event_id_jet_py_std,event_id_jet_pz_mean,event_id_jet_pz_max,event_id_jet_pz_min,event_id_jet_pz_sum,event_id_jet_pz_std,event_id_jet_energy_mean,event_id_jet_energy_max,event_id_jet_energy_min,event_id_jet_energy_sum,event_id_jet_energy_std,event_id_jet_mass_mean,event_id_jet_mass_max,event_id_jet_mass_min,event_id_jet_mass_sum,event_id_jet_mass_std,event_id_jet_energy/num_mean,event_id_jet_energy/num_max,event_id_jet_energy/num_min,event_id_jet_energy/num_sum,event_id_jet_energy/num_std,event_id_jet_mass/num_mean,event_id_jet_mass/num_max,event_id_jet_mass/num_min,event_id_jet_mass/num_sum,event_id_jet_mass/num_std,event_id_jet_px/num_mean,event_id_jet_px/num_max,event_id_jet_px/num_min,event_id_jet_px/num_sum,event_id_jet_px/num_std,event_id_jet_py/num_mean,event_id_jet_py/num_max,...,event_id_jet_id_particle_energy_rel_mean_mean,event_id_jet_id_particle_energy_rel_mean_max,event_id_jet_id_particle_energy_rel_mean_min,event_id_jet_id_particle_energy_rel_mean_sum,event_id_jet_id_particle_energy_rel_mean_std,event_id_jet_id_particle_energy_rel_max_mean,event_id_jet_id_particle_energy_rel_max_max,event_id_jet_id_particle_energy_rel_max_min,event_id_jet_id_particle_energy_rel_max_sum,event_id_jet_id_particle_energy_rel_max_std,event_id_jet_id_particle_energy_rel_min_mean,event_id_jet_id_particle_energy_rel_min_max,event_id_jet_id_particle_energy_rel_min_min,event_id_jet_id_particle_energy_rel_min_sum,event_id_jet_id_particle_energy_rel_min_std,event_id_jet_id_particle_energy_rel_sum_mean,event_id_jet_id_particle_energy_rel_sum_max,event_id_jet_id_particle_energy_rel_sum_min,event_id_jet_id_particle_energy_rel_sum_sum,event_id_jet_id_particle_energy_rel_sum_std,event_id_jet_id_particle_energy_rel_std_mean,event_id_jet_id_particle_energy_rel_std_max,event_id_jet_id_particle_energy_rel_std_min,event_id_jet_id_particle_energy_rel_std_sum,event_id_jet_id_particle_energy_rel_std_std,event_id_jet_id_particle_R_mean_mean,event_id_jet_id_particle_R_mean_max,event_id_jet_id_particle_R_mean_min,event_id_jet_id_particle_R_mean_sum,event_id_jet_id_particle_R_mean_std,event_id_jet_id_particle_R_max_mean,event_id_jet_id_particle_R_max_max,event_id_jet_id_particle_R_max_min,event_id_jet_id_particle_R_max_sum,event_id_jet_id_particle_R_max_std,event_id_jet_id_particle_R_min_mean,event_id_jet_id_particle_R_min_max,event_id_jet_id_particle_R_min_min,event_id_jet_id_particle_R_min_sum,event_id_jet_id_particle_R_min_std,event_id_jet_id_particle_R_sum_mean,event_id_jet_id_particle_R_sum_max,event_id_jet_id_particle_R_sum_min,event_id_jet_id_particle_R_sum_sum,event_id_jet_id_particle_R_sum_std,event_id_jet_id_particle_R_std_mean,event_id_jet_id_particle_R_std_max,event_id_jet_id_particle_R_std_min,event_id_jet_id_particle_R_std_sum,event_id_jet_id_particle_R_std_std
0,00000fdcf3f1454f962edf1cc6a2f64b,5.0,3,19.0,33,11,57,12.165525,-0.41938,79.114197,-86.136002,-1.25814,82.798431,-2.203573,10.598800,-9.941530,-6.61072,11.167477,-2.127223,13.269100,-15.038000,-6.381670,14.316299,62.311333,87.154503,18.139099,186.934006,38.353500,7.282557,10.11920,2.72110,21.847670,3.989261,3.523362,6.280031,1.649009,10.570086,2.438330,0.415630,0.692875,0.247373,1.246890,0.241923,3.073286,6.085708,0.523969,9.219858,2.809641,0.649501,0.963527,...,0.066318,0.091547,0.030404,0.198954,0.031941,0.240530,0.286646,0.149281,0.721590,0.079026,0.005137,0.006439,0.003654,0.015410,0.001402,1.003798,1.007018,1.001028,3.011394,0.003020,0.070152,0.095649,0.033280,0.210455,0.032703,0.133672,0.186960,0.100989,0.401015,0.046542,0.264300,0.289909,0.223454,0.792899,0.

# analyse